In [ ]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
import matplotlib.pylab as plt

from sklearn.model_selection import train_test_split

In [ ]:
train_data = pd.read_csv("data/Train.csv")
test_data = pd.read_csv("data/Test.csv")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data.iloc[:,:9], train_data.iloc[:,9:10], test_size=0.2, random_state=2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
def label_encoding(df):
    X_train_encoded = df.filter(items=['Team1', 'Team2', 'Stadium'])
    X_train_encoded['Team1_Venue'] = le.fit_transform(df['Team1_Venue'])
    X_train_encoded['Team2_Venue'] = le.fit_transform(df['Team2_Venue'])
    X_train_encoded['Team1_Innings'] = le.fit_transform(df["Team1_Innings"])
    X_train_encoded['Team2_Innings'] = le.fit_transform(df["Team2_Innings"])
    
    
    return X_train_encoded

In [ ]:
from sklearn import preprocessing

x = X_train.Stadium.values.reshape(-1,1) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_train.Stadium = pd.DataFrame(x_scaled)
X_train.replace(np.NAN, 0, inplace=True)


X_train_encoded = label_encoding(X_train)
X_test_encoded = label_encoding(X_test)

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5,n_estimators=100, random_state=0)
clf.fit(X_train_encoded, y_train)
randomF_prediction = clf.predict(X_test_encoded)

### Guassian NB

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
guassian_prediction = clf.fit(X_train_encoded, y_train).predict(X_test_encoded)

### Gradient Boost

In [ ]:
X_train_encoded.columns

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=0, n_estimators=100,learning_rate=0.1)
clf.fit(X_train_encoded, y_train)
xgboost_prediction = clf.predict(X_test_encoded)

In [ ]:
train_pred = clf.predict(X_train_encoded)

In [ ]:
from sklearn.metrics import mean_squared_log_error
print("Train:",mean_squared_log_error(y_train, train_pred))
print("Test:",mean_squared_log_error(y_test, xgboost_prediction))

In [ ]:
clf.score(X_train_encoded,y_train)

In [ ]:
# Adding quarter month indicator
# X_train.loc[X_train.eval("MonthOfMatch =='Jan' | MonthOfMatch =='Feb' | MonthOfMatch =='Mar'"), "Indicator" ] = 1

In [ ]:
test_encoded = label_encoding(test_data)

In [ ]:
final_prediction = clf.predict_proba(test_encoded)

In [ ]:
final_pred = pd.DataFrame(data=final_prediction, columns=range(0,16))

In [ ]:
final_pred.to_excel('Final_pred_xgboost.xlsx', index=False)